# Программирование на Python

# Текcтовые файлы и основы работы в `pandas`


*Автор: Паршина Анастасия, НИУ ВШЭ (tg: @aaparshina)*

*Дополнила: Лика Капустина, НИУ ВШЭ (tg: @lika_kapustina)*

## Содержание

1. [Ключевое слово `with` и функция `open()`](#par1)
   1. [Открытие файла](#par1.1)
   2. [Обработка информации из файла](#par1.2)
   3. [Запись информации в файл](#par1.3)
2. [Основы работы в `pandas`](#par2)
   1. [Фильтрация, сортировка, группировка, создание новых признаков](#par2.1)
   2. [Меры центральной тенденции и меры разброса](#par2.2)
3. [Начало работы с текстами](#par3)
4. [Дополнительные материалы](#parlast)

Давайте скачаем файлик [по ссылке](https://github.com/aaparshina/PROG_22-23/blob/main/DATA/wine.csv). Для этого нам нужно нажать `Raw` и затем комбинацию `Ctrl+S` или `Cmd+S`.

[Источник данных - Kaggle](https://www.kaggle.com/datasets/jarredpriester/california-wine-production-19802020).

**Производство вин в Калифорнии 1980 - 2020 гг.**

Данные содержат следующие признаки:

* `Year` - Год
* `County` - Округ, где произвели вино
* `CountyCode` - Номер округа
* `HarvestedAcres` - Собранные акры винограда
* `Yield` - Урожайность на акр
* `Production` - Произведенная продукция (в тоннах)
* `Price` - Цена в далларах за тонну продукции

## Ключевое слово `with` и функция `open()` <a name="par1"></a>

Открыть текстовый файл (например, с расширением `.csv` или `.txt`) можно с помощью ключевого слова `with`. О чем тут важно помнить:

+ файл мы кладем в ту же папку, где у нас лежит блокнот (можно и без этого, но тогда легко запутаться в пути к этому файлу на вашем компьютере)
+ название файла указываем полнотью!
+ указываем режим открытия файла
    + `'r'` = read/чтение (он стоит по умолчанию)
    + `'a'` = append/добавление
    + `'w'` = write/запись, создание
    + на самом деле, их больше, но это основные
   
### Открытие файла <a name="par1.1"></a>

Файл у нас большой, поэтому я попрошу напечатать не все строки из него, а только 5.

In [ ]:
line_num = 5 # задаем число строк

# открываем файл с помощью with()
with open('wine.csv', 'r', encoding = 'utf8') as inf:
    for line in inf:
        print(line)

        line_num -= 1
        if line_num == 0:
            break

Year,CountyCode,County,HarvestedAcres,Yield,Production,Price

2020,1,Alameda,2530.0,5.14,13000.0,1497.69

2020,5,Amador,5360.0,2.31,12400.0,1318.31

2020,9,Calaveras,579.0,3.06,1770.0,1325.99

2020,11,Colusa,747.0,6.02,4500.0,684.67



Самое важное, что тут нужно запомнить — в конце каждой строки (абзаца) стоит переход на новую строку `\n`. При обработке данных советую всегда его убирать, лишним не будет. Сделать это можно с помощью метода `.strip()`.

In [ ]:
line_num = 5
with open('wine.csv', 'r', encoding = 'utf8') as inf:
    for line in inf:
        print(line.strip())

        line_num -= 1
        if line_num == 0:
            break

Year,CountyCode,County,HarvestedAcres,Yield,Production,Price
2020,1,Alameda,2530.0,5.14,13000.0,1497.69
2020,5,Amador,5360.0,2.31,12400.0,1318.31
2020,9,Calaveras,579.0,3.06,1770.0,1325.99
2020,11,Colusa,747.0,6.02,4500.0,684.67


Обратите внимание - здесь и далее мы с вами проходимся по строкам в файле `.csv`. Что мы видим выше?

На первой строке находятся названия колонок:
```
Year,CountyCode,County,HarvestedAcres,Yield,Production,Price
```

Когда мы смотрим на следующие строки, мы соотносим записанные значения с этой самой первой колонкой. Например, для первой строки с данными:

```
2020,1,Alameda,2530.0,5.14,13000.0,1497.69
```

Это означает, что в первой строке записаны следующие данные:
* `Year` - `2020`;
* `CountyCode` - `1`;
* `County` – `Alameda`;
* `HarvestedAcres` – `2530.0`;
* `Yield` – `5.14`;
* `Production` – `13000.0`;
* `Price` – `1497.69`.

Но как можно работать с данными, записанными в одной строке? Файл `.csv` расшировывается как `comma separated values` – то есть это тип файлов, в которых значения разделены запятыми. Это означает, что мы можем спокойно разделить наши значения по запятой и получить список.

In [ ]:
# надо лишь применить метод .split(',')
'2020,1,Alameda,2530.0,5.14,13000.0,1497.69'.split(',')

['2020', '1', 'Alameda', '2530.0', '5.14', '13000.0', '1497.69']

Давайте придумаем нам содержательную задачку на этих данных. Например, посчитаем выручку округа (количество продукции в тоннах умножаем на цену за тонну) и посмотрим, какой округ заработал больше остальных в каждом году.

Сохранять все будем в словарь `data`, где ключи — года, а значения — словари (с парами округ-выручка).

In [ ]:
data = {} # создаем пустой словарь

# открываем файл
with open('wine.csv', 'r', encoding = 'utf8') as input_file:
    for line in input_file:
        line = line.strip().split(',') # превращаем строку в список

        if line[0].isdigit(): # если первое значение - это год,
            year = int(line[0]) # выделяем год
            county = line[2] # выделяем округ

            # получаем значение выручки
            if len(line[-2]) == 0 or len(line[-1]) == 0:
                revenue = 0
            else:
                revenue = float(line[-2]) * float(line[-1])

            # обновляем значение в словаре
            data.setdefault(year, {})
            data[year].update({county: revenue})

### Обработка информации из файла <a name="par1.2"></a>

При работе с такими файлами главное — понять, как удобнее считать из него информацию и затем обработать ее.

Вот так выглядит, например, информация за 2020 год:

In [ ]:
# print(data)

print(data[2020])

{'Alameda': 19469970.0, 'Amador': 16347044.0, 'Calaveras': 2347002.3, 'Colusa': 3081015.0, 'ContraCosta': 6829044.3, 'ElDorado': 9663014.4, 'Fresno': 249876600.0, 'Kern': 59667600.00000001, 'Kings': 16954017.0, 'Lake': 52510114.99999999, 'Madera': 136102200.0, 'Marin': 937999.4, 'Mendocino': 82181760.0, 'Merced': 63419040.0, 'Monterey': 105991314.0, 'Napa': 461395463.99999994, 'Nevada': 0, 'Placer': 1964995.7099999997, 'Riverside': 18999981.0, 'Sacramento': 155975350.0, 'SanBenito': 34773072.0, 'SanBernardino': 440000.91, 'SanDiego': 5206013.9, 'SanJoaquin': 340911680.0, 'SanLuisObispo': 218238370.0, 'SanMateo': 0, 'SantaBarbara': 93836004.0, 'SantaClara': 11981970.8, 'SantaCruz': 4814000.0, 'Shasta': 500000.0, 'Solano': 21189056.0, 'Sonoma': 351511840.0, 'Stanislaus': 35557227.0, 'Tehama': 1053997.3599999999, 'Tulare': 35504000.0, 'Yolo': 104950500.0}


А так можно найти округ с максимальной выручкой:

In [ ]:
sorted([(revenue, county) for county, revenue in data[2020].items()])[-1]

(461395463.99999994, 'Napa')

Посмотрим на округа с максимальной выручкой по каждому году. Дополнительно можем учитывать, сколько раз мы втречали каждый округ.

In [ ]:
counties = {}
for year in sorted(data):
    res = sorted([(revenue, county) for county, revenue in data[year].items()])[-1]
    counties.setdefault(res[1], 0)
    counties[res[1]] += 1
    print(f'В {year} году больше всего выручки получил округ {res[1]}')
    print(f'Выручка составила {res[0]} доллара')

В 1980 году больше всего выручки получил округ Fresno
Выручка составила 62483400.0 доллара
В 1981 году больше всего выручки получил округ Fresno
Выручка составила 59860000.0 доллара
В 1982 году больше всего выручки получил округ Napa
Выручка составила 76285051.0 доллара
В 1983 году больше всего выручки получил округ Napa
Выручка составила 63605976.0 доллара
В 1984 году больше всего выручки получил округ Napa
Выручка составила 76796928.0 доллара
В 1985 году больше всего выручки получил округ Napa
Выручка составила 78286204.0 доллара
В 1986 году больше всего выручки получил округ Napa
Выручка составила 85707540.0 доллара
В 1987 году больше всего выручки получил округ Napa
Выручка составила 82147600.0 доллара
В 1988 году больше всего выручки получил округ SanJoaquin
Выручка составила 146273400.0 доллара
В 1989 году больше всего выручки получил округ Napa
Выручка составила 153706293.0 доллара
В 1990 году больше всего выручки получил округ Napa
Выручка составила 140479616.0 доллара
В 1991 г

In [ ]:
res2 = sorted([(v, k) for k, v in counties.items()], reverse = True)
print(res2)

[(30, 'Napa'), (5, 'SanJoaquin'), (4, 'Sonoma'), (2, 'Fresno')]


### Запись информации в файл <a name="par1.3"></a>

А теперь попробуем все, что мы сейчас получили, записать файл. Для этого нам снова понадобится слово `with` и функция `open()`. Назовем наш файл `wine_result.txt`

In [ ]:
with open('wine_result.txt', 'w', encoding = 'utf8') as output_file:
    # скопируем сюда предыдущий код
    for year in sorted(data):
        res = sorted([(revenue, county) for county, revenue in data[year].items()])[-1]

        print(f'В {year} году больше всего выручки получил округ {res[1]}', file = output_file)
        print(f'Выручка составила {res[0]} доллара', file = output_file)

    print('===', file = output_file)
    print('Подведем итог:', file = output_file)
    for county in res2:
        print(f'Округ {county[1]} был лучшим {county[0]} раз/раза', file = output_file)

Тут ячейка ничего не должна нам выдавать, однако(!) обратите внимание: теперь в папке с блокнотом появился файл `wine_result.txt` со всей информацией, которую мы только что туда записали.

<center><b>Задание 1. Работа с файлами.</b></center>

Скачайте файл об [академической успеваемости студентов](https://www.kaggle.com/datasets/bhavikjikadara/student-study-performance) и поместите его в вашу рабочую директорию (папку, в которой открыт этот ноутбук).

В вашем файле будут следующие колонки:
* `"gender"`
* `"race_ethnicity"`
* `"parental_level_of_education"`
* `"lunch"`
* `"test_preparation_course"`
* `"math_score"`
* `"reading_score"`
* `"writing_score"`

Откройте файл с помощью `with()`, **посчитайте и выведите следующие значения:**

* Число девушек, чьи родители получили магистерскую степень;
* Средний балл девушек по математике;
* Средний балл юношей по математике

**А далее напечатайте только те строки из вашего `.csv` файла**, в которых записана информация о девушках, получивших оценку по математике выше среднего балла для девушек *или* информация о юношах, получивших оценку по математике выше среднего балла для юношей.

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

## Основы работы в `pandas` <a name="par2"></a>

В этом разделе мы, очевидно, и близко не сможем разобрать вообще все. Попробуем сформулировать содержательные вопросы к нашим данным и посмотреть, с помощью каких интрументов возможно их проанализировать.

Обратите внимание – сейчас мы с вами будем работать с тем же файлом `.csv` что и ранее. Однако, `pandas` представляет нам более удобные возможности по обработке и анализу наших данных.

In [3]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/aaparshina/PROG_22-23/main/DATA/wine.csv')
data.head() # первые 5 строк файла

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price
0,2020,1,Alameda,2530.0,5.14,13000.0,1497.69
1,2020,5,Amador,5360.0,2.31,12400.0,1318.31
2,2020,9,Calaveras,579.0,3.06,1770.0,1325.99
3,2020,11,Colusa,747.0,6.02,4500.0,684.67
4,2020,13,ContraCosta,1940.0,4.69,9090.0,751.27


### Фильтрация, сортировка, группировка, создание новых признаков <a name="par2.1"></a>

**1. Сколько всего округов, где производят вино? Сколько всего округов когда-либо производило больше 500 000 тонн продукции?**

In [ ]:
len(data['County'].unique()) # .unique() выдает уникальные значения

42

Далее синтаксис такой: `данные[условие, по которому фильтруем][столбец, который забираем]`

In [ ]:
len(data[data['Production'] > 500000]['County'].unique())

3

**2. Сравните производство округа Калаверас (Calaveras) в 2000 и 2020 годах. Верно ли, что цена за тонну продукции увеличилась на 325 долларов?**

In [ ]:
# здесь у нас несколько условий для фильтрации
data[((data['Year'] == 2000) | (data['Year'] == 2020)) & (data['County'] == 'Calaveras')]

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price
2,2020,9,Calaveras,579.0,3.06,1770.0,1325.99
703,2000,9,Calaveras,470.0,3.60,1690.0,1000.00


**3. В каком году округ Колуса (Colusa) произвел наибольшее/наименьшее количество продукции?**

Отсортировать столбец нам поможет метод `.sort_values()` (обязательно в нем прописываем `by = ` т.е. столбец, по которому фильтруем).

Методы `.tail()` и `.head()` выдают нам низ и верх таблицы (мы указали 1, значит выдаст одну строку снизу и одну строку сверху).

In [ ]:
data[data['County'] == 'Colusa'].sort_values(by = 'Production').tail(1)

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price
291,2012,11,Colusa,2450.0,10.29,25200.0,594.56


In [ ]:
data[data['County'] == 'Colusa'].sort_values(by = 'Production').head(1)

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price
3,2020,11,Colusa,747.0,6.02,4500.0,684.67


**4. Создайте столбец с выручкой округа (количество продукции в тоннах умножаем на цену за тонну). Какой округ заработал больше остальных в 1999 году?**

In [ ]:
data['Value'] = data['Production'] * data['Price']
data.head()

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price,Value
0,2020,1,Alameda,2530.0,5.14,13000.0,1497.69,19469970.0
1,2020,5,Amador,5360.0,2.31,12400.0,1318.31,16347044.0
2,2020,9,Calaveras,579.0,3.06,1770.0,1325.99,2347002.3
3,2020,11,Colusa,747.0,6.02,4500.0,684.67,3081015.0
4,2020,13,ContraCosta,1940.0,4.69,9090.0,751.27,6829044.3


По умолчанию `.sort_values()` сортирует от меньшего к большему, но если указать `ascending = False`, то будет наоборот.

In [ ]:
data[data['Year'] == 1999].sort_values(by = 'Value', ascending = False).head(1)

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price,Value
752,1999,77,SanJoaquin,82012.0,6.69,548800.0,523.0,287022400.0


### Меры центральной тенденции и меры разброса <a name="par2.2"></a>

**5. Какова средняя урожайность на акр в 2020 году?**

In [ ]:
data[data['Year'] == 2020]['Yield'].mean()

5.570294117647059

**6. Выберите только 2015 год. Найдите выбросы по переменной, отвечающей за количество собранных акров винограда, используя межквартильный размах. Какие округи попадают в выбросы?**

In [ ]:
data_15 = data[data['Year'] == 2015]
q1 = data_15['HarvestedAcres'].quantile(0.25)
q3 = data_15['HarvestedAcres'].quantile(0.75)

iqr = q3-q1

bottom = q1 - 1.5 * iqr
top = q3 + 1.5 * iqr

data_15[data_15['HarvestedAcres'] > top]

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price,Value
207,2015,77,SanJoaquin,97900.0,6.23,610000.0,576.15,351451500.0


**7. Посчитайте среднее и медиану по цене за тонну до и после удаления выбросов (предыдущий пункт). Как изменились меры?**

In [ ]:
no_outliners = data_15[data_15['HarvestedAcres'] <= top]

print(data_15['HarvestedAcres'].mean(), data_15['HarvestedAcres'].median())
print(no_outliners['HarvestedAcres'].mean(), no_outliners['HarvestedAcres'].median())

15939.657142857142 4120.0
13529.058823529413 3725.0


**8. В каком году и каким округом было произведено максимальное/минимальное количество продукции?**

In [ ]:
data[data['Production'] == data['Production'].max()]

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price,Value
436,2008,19,Fresno,69631.0,14.94,1040100.0,250.93,260992293.0


In [ ]:
data[data['Production'] == data['Production'].min()]

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price,Value
1225,1983,57,Nevada,NaN,NaN,23.0,500.0,11500.0


**9. Верно ли, что округ Фресно (Fresno) с 1980 года стабильно производит продукцию больше медианного показателя?**

In [ ]:
med = data[data['Production'] > data['Production'].median()]
med[med['County'] == 'Fresno']

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price,Value
6,2020,19,Fresno,56900.0,12.13,690000.0,362.14,249876600.0
42,2019,19,Fresno,53500.0,13.83,740000.0,343.65,254301000.0
80,2018,19,Fresno,59500.0,12.92,769000.0,373.14,286944660.0
117,2017,19,Fresno,59700.0,13.32,795000.0,361.02,287010900.0
153,2016,19,Fresno,75500.0,10.00,755000.0,382.85,289051750.0
189,2015,19,Fresno,60200.0,12.79,770000.0,312.55,240663500.0
223,2014,19,Fresno,58700.0,11.24,660000.0,391.95,258687000.0
257,2013,19,Fresno,54571.0,13.12,716000.0,409.68,293330880.0
293,2012,19,Fresno,86700.0,10.38,900000.0,385.48,346932000.0
329,2011,19,Fresno,68900.0,13.13,905000.0,335.50,303627500.0


**10. В столбце Yield заполните пропущенные значения нулями. Затем создайте новый признак со значением 0 или 1, где 0 — урожайность ниже среднего значения, а 1 — урожайность выше среднего. Каких округов больше? Какой округ во все года попадал в категорию 0 (урожайность ниже среднего значения)?**

In [ ]:
data['Yield'] = data['Yield'].fillna(0)
data['Yield_0_1'] = data['Yield'].apply(lambda x: 0 if x < data['Yield'].mean() else 1)
data.head()

,Year,CountyCode,County,HarvestedAcres,Yield,Production,Price,Value,Yield_0_1
0,2020,1,Alameda,2530.0,5.14,13000.0,1497.69,19469970.0,1
1,2020,5,Amador,5360.0,2.31,12400.0,1318.31,16347044.0,0
2,2020,9,Calaveras,579.0,3.06,1770.0,1325.99,2347002.3,0
3,2020,11,Colusa,747.0,6.02,4500.0,684.67,3081015.0,1
4,2020,13,ContraCosta,1940.0,4.69,9090.0,751.27,6829044.3,0


In [ ]:
data['Yield_0_1'].value_counts()

Yield_0_1
0    838
1    477
Name: count, dtype: int64

In [ ]:
data[data['Yield_0_1'] == 0]['County'].value_counts()

County
Napa             41
Sonoma           40
Calaveras        40
SanDiego         39
SanBernardino    39
Amador           38
SanBenito        37
Riverside        37
SantaClara       37
SantaCruz        36
Nevada           35
Mendocino        35
Alameda          34
Lake             34
SantaBarbara     34
ElDorado         32
Mariposa         30
Monterey         29
SanMateo         28
Placer           28
SanLuisObispo    27
Solano           25
Marin            22
Trinity          20
Yuba             14
Shasta           12
ContraCosta       7
Tehama            2
Mono              2
SanLuisObisp      1
SanJoaquin        1
Yolo              1
Sacramento        1
Name: count, dtype: int64

<center><b>Задание 2.</b></center>

Сохраните в переменную `df` датасет [с информацией об академической успеваемости студентов](https://www.kaggle.com/datasets/bhavikjikadara/student-study-performance) и выведите следующие показатели:

In [ ]:
# YOUR CODE HERE

Выведите 5 строк с информацией о девушках, набравших наибольший балл по математике:

In [ ]:
# YOUR CODE HERE

Посчитайте средние значения по математике, чтению и письму для девушек и юношей:

*Nota bene: если вы хотите решить эту задачу элегантно и быстрее, прочитайте про метод [`groupby` в pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)*

In [ ]:
# YOUR CODE HERE

Создайте колонку `final_grade` - сумму значений результатов по математике, чтению и письму. Выведите средние значения по показателю `final_grade` для студентов с разными уровнями образования родителей (колонка `parental_level_of_education`). Отсортируйте эти средние в порядке возрастания.

*Nota bene: если вы хотите решить эту задачу элегантно и быстрее, прочитайте про метод [`groupby` в pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)*

In [ ]:
# YOUR CODE HERE

# Начало работы с текстами <a name="par3"></a>

Загрузим корпус стихов.

In [4]:
poems = pd.read_csv('poems.csv')
poems

,writer,poem,text
0,Лермонтов Михаил Юрьевич,Любовь мертвеца,NaN
1,Лермонтов Михаил Юрьевич,На серебряные шпоры…,На серебряные шпоры\nЯ в раздумии гляжу;\nЗа т...
2,Лермонтов Михаил Юрьевич,Вид гор из степей Козлова,Пилигрим\nАллах ли там среди пустыни\nЗастывши...
3,Лермонтов Михаил Юрьевич,"К (О, не скрывай! Ты плакала об нем…)","О, не скрывай! Ты плакала об нем –\nИ я его лю..."
4,Лермонтов Михаил Юрьевич,"Жалобы турка (письмо к другу, иностранцу)","Ты знал ли дикий край, под знойными лучами,\nГ..."
...,...,...,...
19311,Владислав Фелицианович Ходасевич,Прогулка,"Хорошо, что в этом мире\nЕсть магические ночи,..."
19312,Владислав Фелицианович Ходасевич,Пробочка,Пробочка над крепким иодом!\nКак ты скоро пере...
19313,Владислав Фелицианович Ходасевич,"Друзья, друзья! Быть может, скоро…","Друзья, друзья! Быть может, скоро —\nИ не во с..."
19314,Владислав Фелицианович Ходасевич,"Увы, дитя! Душе неутоленной…","Увы, дитя! Душе неутоленной\nНе снишься ль ты ..."


Предположим, что мы хотим сделать частотный анализ слов в этом корпусе. Для этого нам нужно выделить слова. Это можно сделать с помощью уже знакомой нам функции `split()`, но здесь она представлена в векторизованной версии: `str.split()`

In [5]:
poems["splitted_text"] = poems["text"].str.split()

In [6]:
poems

,writer,poem,text,splitted_text
0,Лермонтов Михаил Юрьевич,Любовь мертвеца,NaN,NaN
1,Лермонтов Михаил Юрьевич,На серебряные шпоры…,На серебряные шпоры\nЯ в раздумии гляжу;\nЗа т...,"[На, серебряные, шпоры, Я, в, раздумии, гляжу;..."
2,Лермонтов Михаил Юрьевич,Вид гор из степей Козлова,Пилигрим\nАллах ли там среди пустыни\nЗастывши...,"[Пилигрим, Аллах, ли, там, среди, пустыни, Зас..."
3,Лермонтов Михаил Юрьевич,"К (О, не скрывай! Ты плакала об нем…)","О, не скрывай! Ты плакала об нем –\nИ я его лю...","[О,, не, скрывай!, Ты, плакала, об, нем, –, И,..."
4,Лермонтов Михаил Юрьевич,"Жалобы турка (письмо к другу, иностранцу)","Ты знал ли дикий край, под знойными лучами,\nГ...","[Ты, знал, ли, дикий, край,, под, знойными, лу..."
...,...,...,...,...
19311,Владислав Фелицианович Ходасевич,Прогулка,"Хорошо, что в этом мире\nЕсть магические ночи,...","[Хорошо,, что, в, этом, мире, Есть, магические..."
19312,Владислав Фелицианович Ходасевич,Пробочка,Пробочка над крепким иодом!\nКак ты скоро пере...,"[Пробочка, над, крепким, иодом!, Как, ты, скор..."
19313,Владислав Фелицианович Ходасевич,"Друзья, друзья! Быть может, скоро…","Друзья, друзья! Быть может, скоро —\nИ не во с...","[Друзья,, друзья!, Быть, может,, скоро, —, И, ..."
19314,Владислав Фелицианович Ходасевич,"Увы, дитя! Душе неутоленной…","Увы, дитя! Душе неутоленной\nНе снишься ль ты ...","[Увы,, дитя!, Душе, неутоленной, Не, снишься, ..."


Удалим пустые значения.

In [7]:
poems = poems.dropna()

А также приведём слова к единому регистру и удалим знаки препинания (нам хочется, чтобы слова "Там" и "там" считались одним словом, а не разными, а также чтобы знаки препинания не были частью слов).

Пропишем функцию.

In [8]:
def normalize(l):
  punct = [".", ",", "!", "-", ":", "?", "'", ";"]
  for i in range(len(l)):
    l[i] = l[i].lower()
    for symbol in punct:
      if symbol in l[i]:
        l[i] = l[i].replace(symbol, '')
  return l

In [9]:
poems

,writer,poem,text,splitted_text
1,Лермонтов Михаил Юрьевич,На серебряные шпоры…,На серебряные шпоры\nЯ в раздумии гляжу;\nЗа т...,"[На, серебряные, шпоры, Я, в, раздумии, гляжу;..."
2,Лермонтов Михаил Юрьевич,Вид гор из степей Козлова,Пилигрим\nАллах ли там среди пустыни\nЗастывши...,"[Пилигрим, Аллах, ли, там, среди, пустыни, Зас..."
3,Лермонтов Михаил Юрьевич,"К (О, не скрывай! Ты плакала об нем…)","О, не скрывай! Ты плакала об нем –\nИ я его лю...","[О,, не, скрывай!, Ты, плакала, об, нем, –, И,..."
4,Лермонтов Михаил Юрьевич,"Жалобы турка (письмо к другу, иностранцу)","Ты знал ли дикий край, под знойными лучами,\nГ...","[Ты, знал, ли, дикий, край,, под, знойными, лу..."
5,Лермонтов Михаил Юрьевич,К кн. Л. Г-ой,Когда ты холодно внимаешь\nРассказам горести ч...,"[Когда, ты, холодно, внимаешь, Рассказам, горе..."
...,...,...,...,...
19311,Владислав Фелицианович Ходасевич,Прогулка,"Хорошо, что в этом мире\nЕсть магические ночи,...","[Хорошо,, что, в, этом, мире, Есть, магические..."
19312,Владислав Фелицианович Ходасевич,Пробочка,Пробочка над крепким иодом!\nКак ты скоро пере...,"[Пробочка, над, крепким, иодом!, Как, ты, скор..."
19313,Владислав Фелицианович Ходасевич,"Друзья, друзья! Быть может, скоро…","Друзья, друзья! Быть может, скоро —\nИ не во с...","[Друзья,, друзья!, Быть, может,, скоро, —, И, ..."
19314,Владислав Фелицианович Ходасевич,"Увы, дитя! Душе неутоленной…","Увы, дитя! Душе неутоленной\nНе снишься ль ты ...","[Увы,, дитя!, Душе, неутоленной, Не, снишься, ..."


И используем модуль Counter для подсчёта частот.

In [10]:
from collections import Counter
words = []
for poem in poems["splitted_text"]:
  words += normalize(poem)
Counter(words)

Counter({'на': 43740,
         'серебряные': 50,
         'шпоры': 26,
         'я': 53555,
         'в': 98576,
         'раздумии': 7,
         'гляжу': 277,
         'за': 14836,
         'тебя': 7675,
         'скакун': 13,
         'мой': 8977,
         'скорый': 14,
         'бока': 114,
         'твои': 1654,
         'дрожу': 68,
         'наши': 986,
         'предки': 38,
         'их': 8583,
         'не': 68344,
         'знали': 188,
         'и': 155280,
         'гарцуя': 1,
         'средь': 1022,
         'степей': 110,
         'толстой': 58,
         'плеткой': 3,
         'погоняли': 3,
         'недоезжаных': 1,
         'коней': 237,
         'но': 21531,
         'с': 36800,
         'успехом': 36,
         'просвещенья': 46,
         'вместо': 1217,
         'грубой': 29,
         'старины': 84,
         'введены': 4,
         'изобретенья': 3,
         'чужеземной': 1,
         'стороны': 234,
         'наше': 424,
         'время': 1825,
         'кормят': 7,


Всё ли нам нравится в получившемся словаре?

## Дополнительные материалы <a name="parlast"></a>

+ Про открытие файлов в Python [PythonRU](https://pythonru.com/osnovy/fajly-v-python-vvod-vyvod)
+ Как работают кодировки текста. [Статья на Хабр](https://habr.com/ru/post/478636/)
+ [Документация `pandas`](https://pandas.pydata.org/docs/reference/general_functions.html)